Link to Github repository: https://github.com/fridamariejoergensen/CSS.git <br>
### Contribution Statement

We, Frida(s206182), Cecilie(sXXXXXX), and Marie(sXXXXXX), collaborated on Assignment 1 in Jupyter Notebook and made the following contributions:

- Cecilie focused on this

- Frida did this

- Marie did this



# Assignment 1

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import pickle
import logging
import os
import time


## Part 1: Using web-scraping to gather data

### 2019 edition

In [ ]:
## Poster
LINK = "https://2019.ic2s2.org/posters/"
r = requests.get(LINK) 
soup = BeautifulSoup(r.content)

In [ ]:
listul = soup.find("div",{"class":"col-md-8 page-content-wrap"})
listli = listul.find_all("li")

allnames = []
for i in range(len(listli)):
    if listli[i].find("strong"):
        listli[i].strong.extract()
    if listli[i].find("span"):
        listli[i].span.extract()
    if listli[i].find("br"):
        listli[i].br.extract()
        
    names = listli[i].text.replace("\n","").replace(" and ",",").replace(", ",",")
    allnames = allnames + names.split(",")
    


un2019 = np.unique(np.array(allnames))
len(un2019)

There was ?? unique researchers that made poster presentations in 2019  

In [6]:
#### Oral
LINK = 'https://2019.ic2s2.org/oral-presentations/'
r = requests.get(LINK)
all_oral = BeautifulSoup(r.content)

In [ ]:
# extract the specific element of the site that contains the names
paragraphs = all_oral.find_all("p", {"class":""})[3:39]

In [ ]:
names = []
for p in paragraphs:
    text = p.text.strip()
    lines = text.split(' – ')
    names.extend([line.split('.')[0] for line in lines[1:]])

In [ ]:
# code that extracts list elements that contain numbers.

def remove_elements_with_numbers(names):
    return [i for i in names if not any(j.isdigit() for j in i)]

p = remove_elements_with_numbers(names)

In [2]:
newlist = [word for line in p for word in line.split(',')]
newlist.remove('No Presentation')
final_list = np.asarray(newlist)
print(len(np.unique(final_list)))

NameError: name 'p' is not defined

There was ?? unique researchers that made oral presentations in 2019  

In [5]:
# Combine the two lists of poster and oral presentators
combined_list = np.concatenate((final_list, un2019))
# Find the length of unique combined list that attended the conference in 2019 
print(len(np.unique(combined_list)))

NameError: name 'np' is not defined

In total there was ?? unique researchers you got in 2019

Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?

## Part 2: Getting data from the Semantic Scholar API
Consider the list of author ids you have found in Week 2, Part 3, first excercise. For each author, use the Academic Graph API to find: <br>
- their aliases <br>
- their name <br>
- their papers <br>

(**Share the number of authors you will use as starting point in this exercises. Add a comment clarifying how many IC2S2 editions you included and if the collaborators were included or not!!)

Create three dataframe to store the data you have collected:

**Author dataset**: in the author dataset, one raw is one unique author, and each row contains the following information: <br>
- authorId: (str) the id of the author <br>
- name: (str) the name of the author <br> 
- aliases: (list) the aliases of the author <br>
- citationCount: (int) the total number of citations received by an author <br>
- field: (str) the s2FieldsOfStudy that occurs most times across an author's papers (you should first obtain the category for each s2FieldsOfStudy) 



**Paper dataset**: in the paper dataset, one row is one unique paper, and each row contains the following information: <br>
- paperId: (str) the id of the paper <br>
- title: (str) the title of the paper <br> 
- year: (int) the year of publication <br>
- externalId.DOI: (str) the DOI of the paper <br>
- citationCount: (int) the number of citations <br>
- fields: (list) the fields included in the paper (you should first obtain the category for each s2FieldsOfStudy) <br>
-  authorIds: (list) this is a list of author Ids, including all the authors of this paper that are in our author dataset



**Paper abstract dataset**: in the paper abstract dataset, one row is one unique paper, and each row contains the following information: <br>
- paperId: (str) the id of the paper <br>
- abstract: (str) the abstract of the paper


In [ ]:
#Setting up logging of events and errors:
open('collectingData.log','a+')
logging.basicConfig(filename='collectingData.log', 
filemode='a',
level=logging.INFO, 
format='%(asctime)s | %(name)s | %(levelname)s | %(message)s')


In [10]:
# Code to load pickle with all names of conference people and their co-authors:
with open("pickles/all_names","rb") as fp:
    all_names = pickle.load(fp)
print("We have collected {} names.".format(len(all_names.values())))

We have collected 120065 names.


The following is code that creates an object with the authorIds and initializes the whole setup of the dataframes etc.  Then the functions in the object can send requests to the Semantic Scholar API to build up the dataframes.

In [11]:
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"

author_url = BASE_URL + VERSION + "author/batch"

class compSciData:
    def __init__(self,authorIds,picklePath="pickles"):
        self.authorIds = authorIds
        self.authordf = pd.DataFrame(columns = ['authorId','name', 'aliases','citationCount','field'])
        self.paperdf = pd.DataFrame(columns = ['paperId','title', 'year','externalId.DOI','citationCount','fields','authorIds'])
        self.abstractdf = pd.DataFrame(columns = ['paperId','abstract'])    
        self.authordict= {}
        self.paperdict = {}
        self.abstractdict = {}
        self.checkedAuthors = {}
        self.faultyAuthors = {}
        self.picklePath = picklePath
        self.authorURL = BASE_URL + VERSION + "author/batch"
        

    def loadPickles(self):
        if os.path.isfile(self.picklePath+"/finalAuthorDf"):
            self.authordf = pickle.load( open(self.picklePath+"/finalAuthorDf", "rb" ) )
            print("Opened pickle with author dataframe.")
            logging.info("Opened pickle with author dataframe.")

        if os.path.isfile(self.picklePath+"/finalPaperDf"):
            self.paperdf = pickle.load( open( self.picklePath+"/finalPaperDf", "rb" ) )
            print("Opened pickle with paper dataframe.")
            logging.info("Opened pickle with paper dataframe.")

        if os.path.isfile(self.picklePath+"/finalAbstractDf"):
            self.abstractdf = pickle.load( open( self.picklePath+"/finalPaperDf", "rb" ) )
            print("Opened pickle with abstract dataframe.")
            logging.info("Opened pickle with abstract dataframe.")

        if os.path.isfile(self.picklePath+"/checkedAuthors"):
            self.checkedAuthors = pickle.load( open( self.picklePath+"/checkedAuthors", "rb" ) )
            print("Opened pickle with checked authors of length {}.".format(len(self.checkedAuthors)))
            logging.info("Opened pickle with checked authors of length {}.".format(len(self.checkedAuthors)))


    def handleRequests(self,cleanAuthBatch):
        #Send request for the given ids:
        params = {"ids":cleanAuthBatch,
                                "limit":10000}
        r = requests.post(self.authorURL+"?fields=aliases,name,papers,papers.title,papers.abstract,papers.citationCount,"
        +"papers.s2FieldsOfStudy,papers.year,papers.externalIds,papers.authors", json=params)
        data = r.json()
        
        # If data returns message, there is some error, which is likely either too many requests or an internal server error
        if type(data)==list:
            self.sortAuthorDataInDataframesDict(data,cleanAuthBatch)
            # Sleep three secs per request to not exceed 100 requests per 5 min.
            time.sleep(3)
        elif 'message' in data.keys():
            print("Request returned a message: "+data['message'])
            logging.warning("Request returned a message: "+data['message'])
            if data['message']=='Internal server error' or data['message']=='Endpoint request timed out':
                # Sleep three secs per request to not exceed 100 requests per 5 min.
                time.sleep(3)
                self.findFaultyData(cleanAuthBatch)
            else:
                #data['message']=='Something with too many requests':
                time.sleep(30)
                #To be written
                pass

    def findFaultyData(self, cleanAuthBatch):
        batchLength = len(cleanAuthBatch)
        print("Initiating search for faulty data with batchLength {}.".format(batchLength))
        logging.info("Initiating search for faulty data with batchLength {}.".format(batchLength))
        if batchLength>3:
            middleindex = batchLength//2
            halves = [cleanAuthBatch[:middleindex],cleanAuthBatch[middleindex:]]
            for half in halves:
                self.handleRequests(half)
        elif batchLength>1:
            halves = [[id] for id in cleanAuthBatch]
            for half in halves:
                self.handleRequests(half)
        else:
            print("Faulty id: Found that author with id {} returns a server error upon request.".format(cleanAuthBatch))
            logging.warning("Faulty id: Found that author with id {} returns a server error upon request.".format(cleanAuthBatch))
            #Save in the checked authors list, so we don't check author again.
            self.checkedAuthors[cleanAuthBatch[0]]=1 
            self.faultyAuthors[cleanAuthBatch[0]]="error"          

    
    def sortAuthorDataInDataframesDict(self,data,cleanAuthBatch):
        # Takes data object containing the info returned from a request with authorIds. Sorts the data
        # into the dataframes contained in self.
        print("Initiating sortAuthorDataInDataframesDict")
        logging.info("Initiating sortAuthorDataInDataframesDict")

        # Look at every author:
        for i in range(len(data)):
            # Only proceed if the request for current author did return something, aka didn't return None
            if data[i]:
                authorData = data[i]
                papers = authorData['papers']
                #initialize variables for later:
                fields = []
                citationCount = 0
                
                #Look at everyone of their papers:
                for n in range(len(papers)):
                    paper = papers[n]
                    paper_fields = []
                    for s2 in paper['s2FieldsOfStudy']:
                        paper_fields.append(s2['category'])
                    fields=fields+paper_fields
                    citationCount += paper['citationCount']

                    paper_authors = []
                    for auth in paper['authors']:
                        paper_authors.append(auth['authorId'])


                    self.paperdict[paper['paperId']] = {'paperId':paper['paperId'],'title':paper['title'], 'year':paper['year'],
                    'externalId.DOI': paper['externalIds'].get('DOI'),'citationCount':paper['citationCount'],'fields':[paper_fields],
                    'authorIds':[paper_authors]}

                    self.abstractdict[paper['paperId']] = {'paperId':paper['paperId'],'abstract':paper['abstract']}

                self.authordict[authorData['authorId']] = {'authorId' : authorData['authorId'], 'name' : authorData['name'], 
                'aliases' : [authorData['aliases']],'citationCount':citationCount,'field': max(fields,key=fields.count) if fields else None}

                # Add authorId to dict with checked authors:
                self.checkedAuthors[authorData['authorId']]=1

            else:
                print("Author with id {} returned a Nonetype object.".format(cleanAuthBatch[i]))
                logging.info("Author with id {} returned a Nonetype object.".format(cleanAuthBatch[i]))
                self.checkedAuthors[cleanAuthBatch[i]]=1
                self.faultyAuthors[cleanAuthBatch[i]]="none"


    def findData(self, batchsize=100):
        # Avoid batchsize errors:
        if batchsize>100:
            print("Batchsize can maximum be 100 because we are requesting fields=papers attributes.")
            logging.error("Batchsize can maximum be 100 because we are requesting fields=papers attributes.")
            return
        
        # Find papers for every name:
        batches = math.floor(len(self.authorIds)/batchsize)
        for k in range(batches+1):
            #Calculate indexes for next batch:
            lowerbound = k*batchsize
            if k == batches:
                upperbound = lowerbound+len(self.authorIds)%batchsize
            else:
                upperbound = lowerbound+batchsize
            
            #Remove all authors that have already been checked:
            authBatch = self.authorIds[lowerbound:upperbound]
            cleanAuthBatch = []
            for id in authBatch:
                if id not in self.checkedAuthors:
                    cleanAuthBatch.append(id)

            #Only proceed if there are still authors left after cleaning
            if len(cleanAuthBatch)>0:
                
                # Takes authorIds that have not yet been used and handles the request, errorhandling and puts data in 
                # to dataframes in helper functions:
                self.handleRequests(cleanAuthBatch)

                print("Code has processed {} names.".format(upperbound))
                logging.info("Code has processed {} names.".format(upperbound))

                #Save pickles during process as a back up.
                if upperbound%5000==0:
                    self.savePickles(path="pickles/{}".format(upperbound))

    
    def savePickles(self, path="pickles"):
        #Concatenate dataframe so far with the recently collected dicts:
        self.authordf = pd.concat([self.authordf,pd.DataFrame.from_dict(self.authordict,"index")], ignore_index=True)
        self.paperdf = pd.concat([self.paperdf,pd.DataFrame.from_dict(self.paperdict,"index")], ignore_index=True)
        self.abstractdf = pd.concat([self.abstractdf,pd.DataFrame.from_dict(self.abstractdict,"index")], ignore_index=True)

        #Clear dicts, so we don't save them into the dataframe twice:
        self.authordict={}
        self.paperdict={}
        self.abstractdict={}

        #Create folder if it does not exist:
        if not os.path.exists(path):
            os.makedirs(path)

        # Save the pickles:
        pickle.dump(self.authordf,open("{}/finalAuthorDf".format(path),"wb"))
        pickle.dump(self.paperdf,open("{}/finalPaperDf".format(path),"wb"))
        pickle.dump(self.abstractdf,open("{}/finalAbstractDf".format(path),"wb"))
        pickle.dump(self.checkedAuthors,open("{}/checkedAuthors".format(path),"wb"))
        print("Pickles saved succesfully to path {}!".format(path))
        logging.info("Pickles saved succesfully to path {}!".format(path))


In [ ]:
#Takes approx 46 min for 6000 ids
data = compSciData(authorIds=list(all_names.keys())[:10000],picklePath ="pickles")
data.loadPickles()
data.findData(batchsize=100)
data.savePickles()

How long is your final Author dataframe? How long is your final Paper dataframe?

In [ ]:
print("The author dataframe has length:",len(data.authordf))
print("The paper dataframe has length:",len(data.paperdf))

## Part 3: Law of large numbers.